<a href="https://colab.research.google.com/github/donghithanh/Cifar100Classification/blob/main/Cifar100_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

In [2]:
transform = transforms.Compose([transforms.RandomCrop(32, padding=4),transforms.RandomHorizontalFlip(),transforms.ToTensor()])
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,shuffle=True, num_workers=4)

t = transforms.Compose([transforms.ToTensor()])
testset = torchvision.datasets.CIFAR100(root='./data', train=False,download=True, transform=t)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,shuffle=False, num_workers=4)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./data/cifar-100-python.tar.gz to ./data


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Files already downloaded and verified


In [3]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Constraints for layer 1
        self.conv1 = nn.Conv2d(in_channels= 3, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.batch1 = nn.BatchNorm2d(16)
        self.relu1 = nn.ReLU()  # n, 32, 32, 16
        # Constraints for layer 2
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride = 1)
        self.batch2 = nn.BatchNorm2d(32)
        self.relu2 = nn.ReLU()  # n, 30, 30, 32
        # Constraints for layer 3
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride = 1)
        self.batch3 = nn.BatchNorm2d(64)
        self.relu3 = nn.ReLU()  # n, 28, 28, 64
        self.pool3 = nn.MaxPool2d(2,2) # n, 14, 14, 128
        # Constraints for layer 4
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride = 1)
        self.batch4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU()  # n, 12, 12, 128
        # Constraints for layer 5
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride = 1)
        self.batch5 = nn.BatchNorm2d(128)
        self.relu5 = nn.ReLU()     # n, 10, 10, 128
        

        # Defining the Linear layer
        self.fc1 = nn.Linear(128*10*10, 2560)
        self.fc2 = nn.Linear(2560, 100)

    def forward(self, x):
        # -> n, 3, 32, 32
        out = self.conv1(x)
        out = self.batch1(out)
        out = self.relu1(out)           

        out = self.conv2(out)
        out = self.batch2(out)
        out = self.relu2(out)              

        out = self.conv3(out)
        out = self.batch3(out)
        out = self.relu3(out)                   
        out = self.pool3(out)   

        out = self.conv4(out)
        out = self.batch4(out)
        out = self.relu4(out)   

        out = self.conv5(out)
        out = self.batch5(out)
        out = self.relu5(out)

        out = out.view(out.size(0), -1)      
        # Linear Layer
        out = self.fc1(out)
        out = self.fc2(out)
                 # -> n, 100
        return out

In [4]:
model = ConvNet()
model.cuda()

ConvNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (batch1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (batch2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (batch3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu4): ReLU()
  (conv5): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1))
  (batch5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu5): ReLU()
  (fc1): Linear(in_features=12

In [ ]:
model = ConvNet()
model.cuda()

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

num_epochs = 70
logs = []
for epoch in range(num_epochs):
    total_right = 0
    total = 0
    
    for data in trainloader:
        inputs, labels = data
        inputs, labels = Variable(inputs).cuda(),Variable(labels).cuda()
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        
        predicted = outputs.data.max(1)[1]
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()
        
    print('Epoch {}, Training loss {}, Val accuracy {}'.format(epoch+1,loss,total_right/total))
    logs.append('Epoch {}, Training loss: {}, Val accuracy: {}'.format(epoch, loss, total_right/total))
    if (epoch+1)%5==0:
        torch.save(model,'prt_cnn_cifar100.ckpt')
    total_right = 0
    total = 0

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 1, Training loss 4.041775703430176, Val accuracy 0.07553999871015549
Epoch 2, Training loss 3.530397415161133, Val accuracy 0.17712000012397766
Epoch 3, Training loss 2.936615467071533, Val accuracy 0.2406199872493744
Epoch 4, Training loss 2.704740285873413, Val accuracy 0.2942799925804138
Epoch 5, Training loss 3.0999233722686768, Val accuracy 0.3340799808502197
Epoch 6, Training loss 2.4535648822784424, Val accuracy 0.36381998658180237
Epoch 7, Training loss 2.1478543281555176, Val accuracy 0.3912599980831146
Epoch 8, Training loss 2.2048027515411377, Val accuracy 0.40803998708724976
Epoch 9, Training loss 2.3172311782836914, Val accuracy 0.4260599911212921
Epoch 10, Training loss 2.0083649158477783, Val accuracy 0.44001999497413635
Epoch 11, Training loss 2.1557674407958984, Val accuracy 0.45715999603271484
Epoch 12, Training loss 1.8109962940216064, Val accuracy 0.47247999906539917
Epoch 13, Training loss 1.8725793361663818, Val accuracy 0.4784799814224243
Epoch 14, Training

In [ ]:
with open('logs_cnn.txt', 'w') as f:
    for line in logs:
        f.write(line)
        f.write('\n')

In [ ]:
my_model = torch.load('prt_cnn_cifar100.ckpt')

total_right = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images,labels = data
        images, labels = Variable(images).cuda(),Variable(labels).cuda()
        outputs = my_model(images)

        predicted = outputs.data.max(1)[1]
        total += labels.size(0)
        total_right += (predicted == labels.data).float().sum()

print("Test accuracy: %d" % (100*total_right/total))

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Test accuracy: 56


In [5]:
from torchsummary import summary
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ConvNet().to(device)
summary(model,input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             448
       BatchNorm2d-2           [-1, 16, 32, 32]              32
              ReLU-3           [-1, 16, 32, 32]               0
            Conv2d-4           [-1, 32, 30, 30]           4,640
       BatchNorm2d-5           [-1, 32, 30, 30]              64
              ReLU-6           [-1, 32, 30, 30]               0
            Conv2d-7           [-1, 64, 28, 28]          18,496
       BatchNorm2d-8           [-1, 64, 28, 28]             128
              ReLU-9           [-1, 64, 28, 28]               0
        MaxPool2d-10           [-1, 64, 14, 14]               0
           Conv2d-11          [-1, 128, 12, 12]          73,856
      BatchNorm2d-12          [-1, 128, 12, 12]             256
             ReLU-13          [-1, 128, 12, 12]               0
           Conv2d-14          [-1, 128,

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
